# Physical Model

\begin{align}
\partial_{t}\left\langle \hat{\sigma}_{1}^{+}\right\rangle  & =-\frac{2{\rm i}}{\hbar}\left(\overrightarrow{h_{1}}\cdot\left(\left\langle \hat{\sigma}_{1}^{z}\right\rangle ,{\rm i}\left\langle \hat{\sigma}_{1}^{z}\right\rangle ,-\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)-2\left(1+{\rm i}\right)\left(A_{21}+A_{12}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle \left({\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{2}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-4\left(\gamma_{+}+\gamma_{-}+\gamma_{z}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle \left\langle \hat{\sigma}_{2}^{z}\right\rangle ,\\
\partial_{t}\left\langle \hat{\sigma}_{1}^{z}\right\rangle  & =\frac{1}{\hbar}\left(\overrightarrow{h_{1}}\cdot\left({\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right),-{\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right),0\right)+\left({\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)-{\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)\right)\left(A_{21}+A_{12}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{2}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-8\left(\gamma_{+}+\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{z}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{z}\right\rangle \left\langle \hat{\sigma}_{2}^{z}\right\rangle +6\left(\gamma_{+}-\gamma_{-}\right),\\
\partial_{t}\left\langle \hat{\sigma}_{2}^{+}\right\rangle  & =-\frac{2{\rm i}}{\hbar}\left(\overrightarrow{h_{2}}\cdot\left(\left\langle \hat{\sigma}_{2}^{z}\right\rangle ,{\rm i}\left\langle \hat{\sigma}_{2}^{z}\right\rangle ,-\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)-2\left(1+{\rm i}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle \left(A_{12}+A_{21}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{1}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-4\left(\gamma_{+}+\gamma_{-}+\gamma_{z}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle \left\langle \hat{\sigma}_{1}^{z}\right\rangle ,\\
\partial_{t}\left\langle \hat{\sigma}_{2}^{z}\right\rangle  & =\frac{1}{\hbar}\left(\overrightarrow{h_{2}}\cdot\left({\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right),-{\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right),0\right)+\left({\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)-{\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)\right)\left(A_{21}+A_{12}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{1}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-8\left(\gamma_{+}+\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{z}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{z}\right\rangle \left\langle \hat{\sigma}_{1}^{z}\right\rangle +6\left(\gamma_{+}-\gamma_{-}\right).
\end{align}

# Complex ODEINT function

In [1]:
import numpy as np
from scipy.integrate import odeint


def odeintz(func, z0, t, **kwargs):
    """An odeint-like function for complex valued differential equations."""

    # Disallow Jacobian-related arguments.
    _unsupported_odeint_args = ['Dfun', 'col_deriv', 'ml', 'mu']
    bad_args = [arg for arg in kwargs if arg in _unsupported_odeint_args]
    if len(bad_args) > 0:
        raise ValueError("The odeint argument %r is not supported by "
                         "odeintz." % (bad_args[0],))

    # Make sure z0 is a numpy array of type np.complex128.
    z0 = np.array(z0, dtype=np.complex128, ndmin=1)

    def realfunc(x, t, *args):
        z = x.view(np.complex128)
        dzdt = func(z, t, *args)
        # func might return a python list, so convert its return
        # value to an array with type np.complex128, and then return
        # a np.float64 view of that array.
        return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

    result = odeint(realfunc, z0.view(np.float64), t, **kwargs)

    if kwargs.get('full_output', False):
        z = result[0].view(np.complex128)
        infodict = result[1]
        return z, infodict
    else:
        z = result.view(np.complex128)
        return z

# Simulation

### Parameters

In [23]:
"""The user needs to set 2 parameters and the initial conditions of the system."""
"""In our case the parameters are the matrix h and the interaction matrix A."""

h=[[1,2,3],[-2,2,-3]]

A=[[1,2],[3,4]]



In [35]:
def cj(z): 
    return z.conjugate()

def real(z): 
    return z.real

def imag(z): 
    return z.imag


if __name__ == "__main__":

    # Define the right-hand-side of the differential equation.
    def zfunc(z, t, K, L):
        p1, z1, p2, z2 = z
        return [p1 * (K - p2), L - imag(z1) , real(z2),p2+z1] 

    # Set up the inputs and call odeintz to solve the system.
    z0 = np.array([1+2j, 3+4j,1,0])
    t = np.linspace(0, 4, 101)
    K = 3
    L = 1
    z, infodict = odeintz(zfunc, z0, t, args=(K,L), full_output=True)

\begin{align}
\partial_{t}\left\langle \hat{\sigma}_{1}^{+}\right\rangle  & =-\frac{2{\rm i}}{\hbar}\left(\overrightarrow{h_{1}}\cdot\left(\left\langle \hat{\sigma}_{1}^{z}\right\rangle ,{\rm i}\left\langle \hat{\sigma}_{1}^{z}\right\rangle ,-\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)-2\left(1+{\rm i}\right)\left(A_{21}+A_{12}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle \left({\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{2}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-4\left(\gamma_{+}+\gamma_{-}+\gamma_{z}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{+}\right\rangle \left\langle \hat{\sigma}_{2}^{z}\right\rangle ,\\
\partial_{t}\left\langle \hat{\sigma}_{1}^{z}\right\rangle  & =\frac{1}{\hbar}\left(\overrightarrow{h_{1}}\cdot\left({\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right),-{\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right),0\right)+\left({\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)-{\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)\right)\left(A_{21}+A_{12}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{2}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-8\left(\gamma_{+}+\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{z}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{1}^{z}\right\rangle \left\langle \hat{\sigma}_{2}^{z}\right\rangle +6\left(\gamma_{+}-\gamma_{-}\right),\\
\partial_{t}\left\langle \hat{\sigma}_{2}^{+}\right\rangle  & =-\frac{2{\rm i}}{\hbar}\left(\overrightarrow{h_{2}}\cdot\left(\left\langle \hat{\sigma}_{2}^{z}\right\rangle ,{\rm i}\left\langle \hat{\sigma}_{2}^{z}\right\rangle ,-\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)-2\left(1+{\rm i}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle \left(A_{12}+A_{21}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{1}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-4\left(\gamma_{+}+\gamma_{-}+\gamma_{z}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{+}\right\rangle \left\langle \hat{\sigma}_{1}^{z}\right\rangle ,\\
\partial_{t}\left\langle \hat{\sigma}_{2}^{z}\right\rangle  & =\frac{1}{\hbar}\left(\overrightarrow{h_{2}}\cdot\left({\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right),-{\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right),0\right)+\left({\rm Im}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)-{\rm Re}\left(\left\langle \hat{\sigma}_{2}^{+}\right\rangle \right)\right)\left(A_{21}+A_{12}\right)\left({\rm Re}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+{\rm Im}\left(\left\langle \hat{\sigma}_{1}^{+}\right\rangle \right)+\left\langle \hat{\sigma}_{1}^{z}\right\rangle \right)\right)\\
 & \,\,\,\,\,-8\left(\gamma_{+}+\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{z}\right\rangle +8\left(\gamma_{+}-\gamma_{-}\right)\left\langle \hat{\sigma}_{2}^{z}\right\rangle \left\langle \hat{\sigma}_{1}^{z}\right\rangle +6\left(\gamma_{+}-\gamma_{-}\right).
\end{align}